---
在多目标规划问题中，一般不提最优解的概念，只提**满意解**或**有效解**

### 5.5.2 求有效解的几种常用方法
1. 线性加权法
   
   根据目标的重要性确定一个权重，以目标函数的加权平均值为评价函数，使其达到最优
   $$
   \begin{align*}
   &{\rm min}\ \sum\limits_{i=1}^m w_if_i(\boldsymbol x),\\
   &{\rm s.t.}\ {\boldsymbol x}\in {\mathit \Omega}.
   \end{align*}
   $$
2. $\varepsilon$ 约束法
   
   根据决策者的偏好，选择一个主要关注的参考目标，例如 $f_k(\boldsymbol x)$ ，而将其他 $m-1$ 个目标函数放到约束条件中，参数 $\varepsilon_i$ 是决策者对第 $i$ 个目标而言的容许接受阈值
   $$
   \begin{align*}
   &{\rm min}\ f_k(\boldsymbol x),\\
   &{\rm s.t.}
   	\begin{cases}
   	f_i(\boldsymbol x)\leq \varepsilon_i,\quad i=1,2,\cdots,m,\\
   	{\boldsymbol x} \in {\mathit \Omega}.
   	\end{cases}
   \end{align*}
   $$
3. 理想点法
   
   以每个单目标最优值为该目标的理想值，使每个目标函数值与理想值的差的加权平方和最小
   $$
   \mathop{\rm min}\limits_{{\boldsymbol x}\in{\mathit \Omega}}\ \sum\limits_{i=1}^m w_i\left(f_i - f_i^*\right)^2
   $$
   其中 $f_i^*=\mathop {\rm min}\limits_{{\boldsymbol x}\in{\mathit \Omega}}f_i(\boldsymbol x), \quad i=1,2,\cdots,m$，权重进行归一化处理，即满足 $w_i \geq 0,\ \sum\limits_{i=1}^m w_i=1$
4. 优先级法 (序贯解法)
   
   根据目标重要性分成不同优先级，先求优先级高的目标函数最优值，在确保优先级高的目标获得不低于最优值的条件下，再求优先级低的目标函数
   $$
   \begin{align*}
   &f_1^*=\mathop {\rm min}\limits_{{\boldsymbol x}\in{\mathit \Omega}}f_1(\boldsymbol x)\\
   &\\
   &{\rm min}\ f_2(\boldsymbol x),\\
   &{\rm s.t.}
	   \begin{cases}
	   f_i(\boldsymbol x) = f_1^*,\\
	   {\boldsymbol x}\in {\mathit \Omega}
	   \end{cases}\\
   &\cdots{\text 依次递推求解}
   \end{align*}
   $$
   在同一等级的目标可能会有多个，此时可结合加权方法

### 生产与污染问题

In [1]:
import numpy as np
from scipy.optimize import minimize

a = np.array([[0.5, 0.25], [0.2, 0.2], [1, 5]])
b = np.array([8, 4, 72])
minimum_output = 10
profit = np.array([2, 3])
pollution = np.array([1, 2])
c = np.vstack([-profit, pollution])

#### 1. 线性加权法
权重 $w_1, w_2$ 均设为 $0.5$
> 也可采用`cvxpy`求解

In [2]:
w = np.array([0.5, 0.5])

def obj(x):
    return w @ (c @ x)

def ineq(x):
    return [
        b[0] - a[0] @ x,    # b - a @ x 的写法报错：只有大小为 1的 array可以被转换为 Python标量
        b[1] - a[1] @ x,
        b[2] - a[2] @ x,
        np.sum(x) - minimum_output
    ]

cons = [{'type': 'ineq', 'fun': ineq}]
bd = [(0, np.inf) for _ in range(2)]
ret = minimize(obj, np.random.rand(2), constraints=cons, bounds=bd)
print(ret)

     fun: -10.000000149011631
     jac: array([-0.5, -0.5])
 message: 'Optimization terminated successfully'
    nfev: 12
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([10.34757892,  9.65242138])


#### 2. 理想点法

In [3]:
import cvxpy as cp

x = cp.Variable(2, nonneg=True)
cons = [
    a @ x <= b,
    cp.sum(x) >= minimum_output
]

obj1 = cp.Maximize(profit @ x)
prob1 = cp.Problem(obj1, cons)
prob1.solve(solver='GUROBI')
f1star = prob1.value

obj2 = cp.Minimize(pollution @ x)
prob2 = cp.Problem(obj2, cons)
prob2.solve(solver='GUROBI')
f2star = prob2.value

f1star, f2star

(52.99999999999999, 10.0)

In [4]:
obj = cp.Minimize((profit @ x - f1star)**2 + (pollution @ x - f2star)**2)
prob = cp.Problem(obj, cons)
prob.solve(solver='GUROBI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')

最优解为：[13.36  5.28]
最优值为：302.7600000000323


(None, None)

#### 3. 序贯解法
若以利润最大化为一级目标

In [5]:
import cvxpy as cp

x = cp.Variable(2, nonneg=True)
cons = [
    a @ x <= b,
    cp.sum(x) >= minimum_output
]

obj1 = cp.Maximize(profit @ x)
prob1 = cp.Problem(obj1, cons)
prob1.solve(solver='GUROBI')
f1star = prob1.value
f1star

52.99999999999999

In [6]:
cons.append(profit @ x == f1star)
obj = cp.Minimize(pollution @ x)
prob = cp.Problem(obj, cons)
prob.solve(solver='GUROBI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')
print('-'*100)
print(f'利润为：{np.round(f1star, 1)}'), print(f'污染为：{np.round(prob.value, 1)}')

最优解为：[ 7. 13.]
最优值为：32.99999999999999
----------------------------------------------------------------------------------------------------
利润为：53.0
污染为：33.0


(None, None)